In [1]:
import pickle
import os
import numpy as np
import sklearn.metrics as metrics
import matplotlib.pyplot as plt


from tensorflow.keras.callbacks import ModelCheckpoint,CSVLogger
from keras.layers import Activation, Conv1D, Dense, Embedding, Flatten, Input, MaxPooling1D
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.datasets import fetch_20newsgroups
from sklearn.datasets.base import get_data_home
from keras.metrics import categorical_accuracy
from sklearn.model_selection import train_test_split


Using TensorFlow backend.


In [2]:
#Processed Data Frame [ Do Not Use this, Use the Training, Tesing and Validation Data Sets Instead]
data_path = "./Pickles/all_articles_processed.pickle"
with open(data_path, 'rb') as data:
    df = pickle.load(data)
    
    
#TD-IDF Features    
#Training Features
training_features_path = "./Pickles/tdidf_training_features.pickle"
with open(training_features_path, 'rb') as data:
    tdidf_training_features = pickle.load(data)
    
#Training Labels
training_labels_path = "./Pickles/tdidf_training_labels.pickle"
with open(training_labels_path, 'rb') as data:
    tdidf_training_labels = pickle.load(data)
    
#Test Features
test_features_path = "./Pickles/tdidf_test_features.pickle"
with open(test_features_path, 'rb') as data:
    tdidf_test_features = pickle.load(data)
    
#Test Labels
test_labels_path = "./Pickles/tdidf_test_labels.pickle"
with open(test_labels_path, 'rb') as data:
    tdidf_test_labels = pickle.load(data)
    
#Validation Features
test_features_path = "./Pickles/tdidf_validation_features.pickle"
with open(test_features_path, 'rb') as data:
    tdidf_validation_features = pickle.load(data)
    
#Validation Labels
test_labels_path = "./Pickles/tdidf_validation_labels.pickle"
with open(test_labels_path, 'rb') as data:
    tdidf_validation_labels = pickle.load(data)
    
    
#Sequence Vector Features    
#Training Features
training_features_path = "./Pickles/sv_training_features.pickle"
with open(training_features_path, 'rb') as data:
    sv_training_features = pickle.load(data)
    
#Training Labels
training_labels_path = "./Pickles/sv_training_labels.pickle"
with open(training_labels_path, 'rb') as data:
    sv_training_labels = pickle.load(data)
    
#Test Features
test_features_path = "./Pickles/sv_test_features.pickle"
with open(test_features_path, 'rb') as data:
    sv_test_features = pickle.load(data)
    
#Test Labels
test_labels_path = "./Pickles/sv_test_labels.pickle"
with open(test_labels_path, 'rb') as data:
    sv_test_labels = pickle.load(data)
    
#Validation Features
test_features_path = "./Pickles/sv_validation_features.pickle"
with open(test_features_path, 'rb') as data:
    sv_validation_features = pickle.load(data)
    
#Validation Labels
test_labels_path = "./Pickles/sv_validation_labels.pickle"
with open(test_labels_path, 'rb') as data:
    sv_validation_labels = pickle.load(data)

In [3]:
df.head()

,source,title,article,category,length_characters,length_words,category_code
0,The Straits Times,Will Chinese teen icon Wang Yuan's career go u...,wang yuans reputation career danger go smoke ...,Lifestyle,1697,292,3
1,The Straits Times,The most value-for-money CBD set lunches,singapore weekday lunch hour central busines...,Lifestyle,8538,1363,3
2,AsiaOne,How much does adopting a child in Singapore co...,moment melissa hold weekold baby girl arm coul...,Lifestyle,8825,1508,3
3,Channel News Asia,NaN,singapore find favourite carrot cake roti prat...,Lifestyle,3351,546,3
4,The Straits Times,Fans fishing for answers to question of whethe...,water murky whether malaysian singer fish leo...,Lifestyle,1409,245,3


In [4]:
df['category'].unique()

array(['Lifestyle', 'World', 'Technology', 'Business', 'Singapore',
       'Sports'], dtype=object)

In [5]:
len(df)

3000

# GloVe embeddings

In [6]:
texts = df['article'] #Extract text
target = df['category'] # Extract target

In [7]:
print (target[:10])

print (len(texts))
print (len(target))
#print (len(texts[0].split()))
print (texts[0])
print (target[0])
print (df['category'][0])

0    Lifestyle
1    Lifestyle
2    Lifestyle
3    Lifestyle
4    Lifestyle
5    Lifestyle
6    Lifestyle
7    Lifestyle
8    Lifestyle
9    Lifestyle
Name: category, dtype: object
3000
3000
 wang yuans reputation career danger go smoke singer chinese boyband tfboys recently photograph smoke beijing restaurant hashtag wang yuan smoke draw  billion click weibo tuesday may  wang  see top teen icon smoke incident punish accord law statement beijing health supervision authority say     smoke ban inside public indoor space china career risk stall since authorities know crack hard inappropriate celebrity behaviour sport tattoo male artists wear earrings wang face danger blacklist entertainment industry try stub damage issue apology weibo nearly  million followers      incident make deeply reflect behaviour sorry embarrass adverse social impact create post           relate story members popular chinese teen band tfboys fan maternal instinct admirers    hope nobody follow wrongful action apolog

In [8]:
vocab_size = 20000

tokenizer = Tokenizer(num_words=vocab_size) # Setup tokenizer
tokenizer.fit_on_texts(df['article'])
sequences = tokenizer.texts_to_sequences(df['article']) # Generate sequences

In [9]:
print (tokenizer.texts_to_sequences(['Hello King, how are you?']))

print (len(sequences))
print (len(sequences[0]))
print (sequences[0])

[[5467, 1851]]
3000
154
[1693, 7389, 1942, 666, 3529, 23, 1607, 822, 43, 4846, 18849, 688, 2351, 1607, 266, 948, 6640, 1693, 833, 1607, 433, 48, 2780, 1785, 119, 41, 1693, 54, 108, 2123, 3390, 1607, 561, 3874, 76, 387, 246, 266, 339, 6338, 953, 1, 1607, 532, 1052, 93, 2502, 336, 21, 666, 280, 1476, 38, 347, 70, 2503, 749, 4985, 2292, 1381, 434, 15469, 2228, 1804, 788, 15470, 1693, 97, 3529, 3530, 1236, 204, 122, 18850, 811, 133, 2988, 1785, 475, 30, 2352, 561, 7, 2429, 1382, 1381, 2781, 3792, 6063, 144, 489, 287, 49, 120, 53, 80, 393, 672, 43, 2123, 1248, 18849, 290, 18851, 18852, 15471, 723, 3961, 136, 11802, 328, 1715, 12, 7001, 118, 227, 582, 68, 290, 50, 60, 615, 38, 1716, 459, 155, 627, 170, 122, 589, 310, 76, 132, 8, 1693, 3, 1715, 994, 15, 13, 1053, 524, 7839, 181, 439, 8351, 427, 314, 8352, 85, 653, 42, 73, 394]


In [10]:
word_index = tokenizer.word_index
print('Found {:,} unique words.'.format(len(word_index)))

Found 45,521 unique words.


In [11]:
# Create inverse index mapping numbers to words
inv_index = {v: k for k, v in tokenizer.word_index.items()}

# Print out text again
for w in sequences[0]:
    x = inv_index.get(w)
    print(x,end = ' ')

wang yuans reputation career danger go smoke singer chinese boyband tfboys recently photograph smoke beijing restaurant hashtag wang yuan smoke draw billion click weibo tuesday may wang see top teen icon smoke incident punish accord law statement beijing health supervision authority say smoke ban inside public indoor space china career risk stall since authorities know crack hard inappropriate celebrity behaviour sport tattoo male artists wear earrings wang face danger blacklist entertainment industry try stub damage issue apology weibo nearly million followers incident make deeply reflect behaviour sorry embarrass adverse social impact create post   relate story members popular chinese teen band tfboys fan maternal instinct admirers hope nobody follow wrongful action apologise take lesson become better person many fan give second chance since mistake others note walk talk try role model accord global time wang also apologise december last year jump safety fence side road beg question 

In [12]:
# Get the average length of a text
avg = sum(map(len, sequences)) / len(sequences)

# Get the standard deviation of the sequence length
std = np.sqrt(sum(map(lambda x: (len(x) - avg)**2, sequences)) / len(sequences))

avg,std

(249.76766666666666, 173.12584735163657)

In [13]:
print(pad_sequences([[1,2,3]], maxlen=5))
print(pad_sequences([[1,2,3,4,5,6]], maxlen=5))

[[0 0 1 2 3]]
[[2 3 4 5 6]]


In [14]:
max_length = 100
data = pad_sequences(sequences, maxlen=max_length)

In [15]:
from keras.utils import to_categorical
labels = to_categorical(np.asarray(df['Category_code']))
print('Shape of data:', data.shape)
print('Shape of labels:', labels.shape)

print (df['category_code'][0])
print (labels[0])

KeyError: 'Category_code'

In [ ]:
glove_dir = './Input/Glove' # This is the folder with the dataset

embeddings_index = {} # We create a dictionary of word -> embedding

with open(os.path.join(glove_dir, 'glove.6B.100d.txt'), encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0] # The first value is the word, the rest are the values of the embedding
        embedding = np.asarray(values[1:], dtype='float32') # Load embedding
        embeddings_index[word] = embedding # Add embedding to our embedding dictionary

print('Found {:,} word vectors in GloVe.'.format(len(embeddings_index)))

In [ ]:
print (embeddings_index['frog'])
print (len(embeddings_index['frog']))

In [ ]:
print (np.linalg.norm(embeddings_index['man'] - embeddings_index['woman']))
print (np.linalg.norm(embeddings_index['man'] - embeddings_index['cat']))

# https://nlp.stanford.edu/projects/glove/
print (np.linalg.norm(embeddings_index['frog'] - embeddings_index['toad']))
print (np.linalg.norm(embeddings_index['frog'] - embeddings_index['man']))

print (np.linalg.norm(embeddings_index['frog'] - embeddings_index['fog']))

print (np.linalg.norm(embeddings_index['frog'] - embeddings_index['fork']))
print (np.linalg.norm(embeddings_index['frog'] - embeddings_index['skyscraper']))

In [ ]:
embedding_dim = 100 # We use 100 dimensional glove vectors

word_index = tokenizer.word_index
nb_words = min(vocab_size, len(word_index)) # How many words are there actually

embedding_matrix = np.zeros((nb_words, embedding_dim))

# The vectors need to be in the same position as their index. 
# Meaning a word with token 1 needs to be in the second row (rows start with zero) and so on

# Loop over all words in the word index
for word, i in word_index.items():
    # If we are above the amount of words we want to use we do nothing
    if i >= vocab_size: 
        continue
    # Get the embedding vector for the word
    embedding_vector = embeddings_index.get(word)
    # If there is an embedding vector, put it in the embedding matrix
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector

In [ ]:
print (embedding_matrix[100])

In [ ]:
def createModel():
    model = Sequential()
    model.add(Embedding(vocab_size, 
                        embedding_dim, 
                        input_length=max_length, 
                        weights = [embedding_matrix], 
                        trainable = False))
    model.add(Conv1D(128, 3, activation='relu'))
    model.add(MaxPooling1D(3))
    model.add(Conv1D(128, 3, activation='relu'))
    model.add(MaxPooling1D(3))
    model.add(Conv1D(128, 3, activation='relu'))
    model.add(MaxPooling1D(3))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(6, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
    optimizer='rmsprop',
    metrics=['accuracy'])    
    return model

In [ ]:
# Setup the models
model       = createModel() # This is meant for training
modelGo     = createModel() # This is used for final testing

model.summary()

In [ ]:
modelname   = 'Exercise_4_1'
filepath        = modelname + ".hdf5"
checkpoint      = ModelCheckpoint(filepath, 
                                  monitor='val_acc', 
                                  verbose=0, 
                                  save_best_only=True, 
                                  mode='max')

                            # Log the epoch detail into csv
csv_logger      = CSVLogger(modelname +'.csv')
callbacks_list  = [checkpoint,csv_logger]


In [ ]:
trDat, tsDat, trLbl, tsLbl = train_test_split(data, labels, test_size=0.33, random_state=42)

In [ ]:
# model.compile(optimizer='adam',
#               loss='binary_crossentropy',  # https://stackoverflow.com/questions/42081257/keras-binary-crossentropy-vs-categorical-crossentropy-performance
#               metrics=['accuracy'])

# https://stackoverflow.com/questions/42081257/keras-binary-crossentropy-vs-categorical-crossentropy-performance
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#model.fit(data, labels, validation_split=0.2, epochs=10)


model.fit(trDat, 
          trLbl, 
          validation_data=(tsDat, tsLbl), 
          epochs=10, 
          batch_size=128,
          shuffle=True,
          callbacks=callbacks_list)


In [ ]:
predicts    = modelGo.predict(tsDat)

In [ ]:
predout     = np.argmax(predicts,axis=1)
testout     = np.argmax(tsLbl,axis=1)

In [ ]:
labelname   = ['Singapore', 'Sport', 'Lifestyle', 'World', 'Business', 'Tech']

In [ ]:
testScores  = metrics.accuracy_score(testout,predout)
confusion   = metrics.confusion_matrix(testout,predout)

In [ ]:
print("Best accuracy (on testing dataset): %.2f%%" % (testScores*100))
print(metrics.classification_report(testout,predout,target_names=labelname,digits=4))
print(confusion)

In [ ]:
import pandas as pd

records     = pd.read_csv(modelname +'.csv')
plt.figure()
plt.subplot(211)
plt.plot(records['val_loss'])
plt.plot(records['loss'])
plt.yticks([0,0.20,0.40,0.60,0.80,1.00])
plt.title('Loss value',fontsize=12)

ax          = plt.gca()
ax.set_xticklabels([])



plt.subplot(212)
plt.plot(records['val_acc'])
plt.plot(records['acc'])
plt.yticks([0.6,0.7,0.8,0.9,1.0])
plt.title('Accuracy',fontsize=12)
plt.show()


In [ ]:
example = data[400] # get the tokens
print (df['article'][400])

# Print tokens as text
for w in example:
    x = inv_index.get(w)
    print(x,end = ' ')

In [ ]:
# Get prediction
pred = model.predict(example.reshape(1,100))

In [ ]:
# Output predicted category
df['category'][np.argmax(pred)]

# Hyperparameters Optimization

In [ ]:
with open('Models/decisionTree_best_model.pickle', 'wb') as output:
    pickle.dump(best_lrc, output)
    
with open('Models/decisionTree_best_model_details.pickle', 'wb') as output:
    pickle.dump(df_models_lrc, output)